# 10. 워드 임베딩(Word Embedding)

## 12) 문서 임베딩 : 워드 임베딩의 평균(Average Word Embedding)

영화 사이트 IMDB 영화 리뷰 데이터는 리뷰 텍스트에 리뷰가 긍정인 경우 1을, 부정인 경우 0으로 레이블링 한 데이터로 25,000개의 훈련 데이터와 테스트 데이터 25,000개로 구성된 데이터입니다.

### 1. 데이터 로드와 전처리

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

데이터를 로드할 때 파라미터로 num_words를 사용하면 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지를 사용할 것인지를 의미합니다.

In [2]:
vocab_size = 20000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(X_train))
print('테스트용 리뷰 개수 :',len(X_test))

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\seungwon\anaconda3\envs\scratch\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


C:\Users\seungwon\anaconda3\envs\scratch\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


이 데이터는 이미 정수 인코딩까지의 전처리가 진행되어져 있습니다. 

첫번째 리뷰와 첫번째 리뷰의 레이블을 출력해봅시다.

In [5]:
print('훈련 데이터의 첫번째 샘플 :', X_train[0])
print('훈련 데이터의 첫번째 샘플의 레이블 :', y_train[0])

훈련 데이터의 첫번째 샘플 : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
훈련 데이터의 첫번째 샘플의 레이블 : 1


In [6]:
print('훈련용 리뷰의 평규 길이: {}'.format(np.mean(list(map(len, X_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, X_test)), dtype=int)))

훈련용 리뷰의 평규 길이: 238
테스트용 리뷰의 평균 길이: 230


훈련용 리뷰와 테스트용 리뷰의 평균 길이가 각각 238, 230입니다. 평균보다는 큰 수치인 400으로 패딩합니다.

In [7]:
max_len = 400

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
print('X_train의 크기(shape) :', X_train.shape)
print('X_test의 크기(shape) :', X_test.shape)

X_train의 크기(shape) : (25000, 400)
X_test의 크기(shape) : (25000, 400)


### 2. 모델 설계하기

임베딩 벡터를 평균으로 사용하는 모델을 설계해봅시다.

GlobalAveragePooling1D()은 입력으로 들어오는 모든 벡터들의 평균을 구하는 역할을 합니다. Embedding() 다음에 GlobalAveragePooling1D()을 추가하면 해당 문장의 모든 단어 벡터들의 평균 벡터를 구합니다.

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 64

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))

# 모든 단어 벡터의 평균을 구한다.
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('results/embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 9s 11ms/step - loss: 0.6294 - acc: 0.7142 - val_loss: 0.5224 - val_acc: 0.8078

Epoch 00001: val_acc improved from -inf to 0.80780, saving model to embedding_average_model.h5
Epoch 2/10
625/625 [==============================] - 7s 11ms/step - loss: 0.4304 - acc: 0.8582 - val_loss: 0.3849 - val_acc: 0.8588

Epoch 00002: val_acc improved from 0.80780 to 0.85880, saving model to embedding_average_model.h5
Epoch 3/10
625/625 [==============================] - 7s 11ms/step - loss: 0.3190 - acc: 0.8914 - val_loss: 0.3223 - val_acc: 0.8806

Epoch 00003: val_acc improved from 0.85880 to 0.88060, saving model to embedding_average_model.h5
Epoch 4/10
625/625 [==============================] - 7s 11ms/step - loss: 0.2621 - acc: 0.9095 - val_loss: 0.3027 - val_acc: 0.8814

Epoch 00004: val_acc improved from 0.88060 to 0.88140, saving model to embedding_average_model.h5
Epoch 5/10
625/625 [==============================] - 7s 11ms/step - loss: 

In [10]:
loaded_model = load_model('results/embedding_average_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 2s 3ms/step - loss: 0.2936 - acc: 0.8860

 테스트 정확도: 0.8860
